# Moving average:

In [ ]:
ts_plot.plotMovingAverage(ads, 4, plot_intervals=True, plot_anomalies=True)
ts_plot.plotMovingAverage(ads, 12, plot_intervals=True, plot_anomalies=True)
ts_plot.plotMovingAverage(ads, 24, plot_intervals=True, plot_anomalies=True)

In [ ]:
ts_plot.plotMovingAverage(currency, 7, plot_intervals=True, plot_anomalies=True)

# Exponential smoothing

In [ ]:
ts_plot.plotExponentialSmoothing(ads.Ads, [0.3, 0.05])
ts_plot.plotExponentialSmoothing(currency.GEMS_GEMS_SPENT, [0.3, 0.05])

# Double exponential smoothing

In [ ]:
ts_plot.plotDoubleExponentialSmoothing(ads.Ads, alphas=[0.9, 0.02], betas=[0.9, 0.02])
ts_plot.plotDoubleExponentialSmoothing(currency.GEMS_GEMS_SPENT, alphas=[0.9, 0.02], betas=[0.9, 0.02])

# Triple exponential smoothing (Holt-Winters)

# Timeseries crossvalidation

In [ ]:
from sklearn.model_selection import TimeSeriesSplit 

# Finding optimal smoothing parameters
Each of the smoothing parameters has to be $\in [0,1]$, therefore we use truncated Newton conjugate gradient algorithm for minimizing the loss function as it supports constraints on model parameters.

In [ ]:
%%time
data = ads.Ads[:-20] # leave some data for testing

# initializing model parameters alpha, beta and gamma
x = [0, 0, 0] 

# Minimizing the loss function 
opt = minimize(timeseriesCVscore, x0=x, 
               args=(data, mean_squared_log_error), 
               method="TNC", bounds = ((0, 1), (0, 1), (0, 1)))

# Take optimal values...
alpha_final, beta_final, gamma_final = opt.x
print(alpha_final, beta_final, gamma_final)

The resulting smoothing function:

In [ ]:
%%time
model = HoltWinters(data, slen = 24, 
                    alpha = alpha_final, 
                    beta = beta_final, 
                    gamma = gamma_final, 
                    n_preds = 50, scaling_factor = 3)
model.triple_exponential_smoothing()

Plotting the triple smoothing

In [ ]:
ts_trends.plotHoltWinters(ads.Ads)
ts_trends.plotHoltWinters(ads.Ads, plot_intervals=True, plot_anomalies=True)

In [ ]:
Compute deviation for anomaly detection:

In [ ]:
plt.figure(figsize=(25, 5))
plt.plot(model.PredictedDeviation)
plt.grid(True)
plt.axis('tight')
plt.title("Brutlag's predicted deviation");

# Triple smoothing of the currency series
Finding optimal smoothing parameters:

In [ ]:
%%time
data = currency.GEMS_GEMS_SPENT[:-50] 
slen = 30 # 30-day seasonality

x = [0, 0, 0] 

opt = minimize(timeseriesCVscore, x0=x, 
               args=(data, mean_absolute_percentage_error, slen), 
               method="TNC", bounds = ((0, 1), (0, 1), (0, 1))
              )

alpha_final, beta_final, gamma_final = opt.x
print(alpha_final, beta_final, gamma_final)

In [ ]:
%%time
model = HoltWinters(data, slen = slen, 
                    alpha = alpha_final, 
                    beta = beta_final, 
                    gamma = gamma_final, 
                    n_preds = 100, scaling_factor = 3)
model.triple_exponential_smoothing()

In [ ]:
ts_trends.plotHoltWinters(currency.GEMS_GEMS_SPENT)
ts_trends.plotHoltWinters(currency.GEMS_GEMS_SPENT, 
                plot_intervals=True, plot_anomalies=True)

In [ ]:
plt.figure(figsize=(25, 5))
plt.plot(model.PredictedDeviation)
plt.grid(True)
plt.axis('tight')
plt.title("Brutlag's predicted deviation");